# <span style="color:blue">Project X4 данные. Часть первая </span>

## Вступление <a class="anchor" id="id_0"></a> 

### Оглавление 

* [Вступление](#id_0)  
  * [Библиотеки](#id_1)
  * [Описание](#id_1a)
  * [Папки с файлами](#id_1b)
  * [Необходимые функции](#id_1c)
* [Получение данных](#id_2)
  * [Создание списка нужных файлов](#id_2a)
  * [Нахождение информации в разных тегах XML на примере аргонского дестроера](#id_2b)
  * [Создание списка путей к конфигурационным файлам кораблей](#id_2b_1)
  * [Создание структуры тегов файла descriptions_eng](#id_2c)
  * [Создание общей таблицы характеристик кораблей по всем файлам](#id_2d)
    * [Создание таблицы](#id_2d_1)
    * [Обработка таблицы на наличие ошибок](#id_2d_2)
  * [Справочная информация](#id_2e)
  * [Нахождение информации о количестве слотов  на кораблях](#id_2f)
  * [Создание выборки кораблей по назначению](#id_2e)
* [Получение информации о модулях корабля](#id_3)
  * [Щиты](#id_3a) 
  * [Двигатели](#id_3b)
* [Сохранение таблиц](#id_4)

### Libraries <a class="anchor" id="id_1"></a> 

In [78]:
import pandas as pd
import os
import re
from bs4 import BeautifulSoup
import lxml
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
#pd.set_option('max_colwidth', 300)

### Описание <a class="anchor" id="id_1a"></a> 

X4: Foundations - космическая "песочница", в которой игрок может строить свои фабрики и верфи, торговать с другими расами, воевать и захватывать корабли и выполнять масштабные сюжетные кампании в динамичном и развивающемся космосе с красивой графикой и эффектами. Игра была разработана кампанией Egosoft в 2018 году и к настоящему моменту имеет 4 дополнения.  
Информация о кораблях собрана для всех объектов.  
Данные о щитах собраны для всех щитов.  
Данные о двигателях собраны для всех двигателей, за исключением двигателей мин, ракет, дронов, скафандра, _xs_объектов. 

### Папки с файлами <a class="anchor" id="id_1b"></a> 

In [79]:
# Каталоги с игровыми объектами
units_folder = r'H:\Steam\X4\Unpacked\assets\units' # каталог с файлами xml, которых сохранена информация о игровых объектах.
boron_dlc_folder = r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units' # каталог с объектами из длс "Kingdom End"
avarice_dlc_folder = r'H:\Steam\X4\Unpacked\extensions\avarice_dlc\assets\units' # каталог с объектами из длс "Tides of Avarice"
terran_dlc_folder = r'H:\Steam\X4\Unpacked\extensions\terran_dlc\assets\units'  # # каталог с объектами из длс "Cradle of humanity"
split_dlc_folder = r'H:\Steam\X4\Unpacked\extensions\split_dlc\assets\units' # каталог с объектами из длс "Split Vendetta"
# Каталоги с текстовой информацией
descriptions_eng = r'H:\Steam\X4\Unpacked\t\0001-l044.xml' #файл, в котором закодирована вся текстовая информация для английской версии
descriptions_ru = r'H:\Steam\X4\Unpacked\t\0001-l007.xml' #файл, в котором закодирована вся текстовая информация для русской версии
# Проблемный файл, который не несет никакой полезной информации, но вызывает кучу ошибок
problemic_file = r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units\size_l\ship_bor_l_miner_solid_01_macro.xml'
#Каталог сохранения. Операции сохранения в ячейках 187-189
save_folder = r'H:\Steam\X4\SavedTables' # Каталог сохранения полученных csv файлов

### Необходимые функции <a class="anchor" id="id_1c"></a> 

In [80]:
#функция для выравнивания списков
def flatten(l):
    return [item for sublist in l for item in sublist]

In [81]:
# Функция для вывода пропущенных значений в датафрейме, просто вызовите ее, указав нужный датафрейм в параметрах
def missing_values_tab(df):
    #Подсчет количества пропущенных значений
    mis_val = df.isnull().sum()
    #Подсчет доли пропущенных значений в колонке в процентах
    mis_val_percent = round(100 * df.isnull().sum() / len(df),2)
    #Объединяем две созданных колонки в один датафрейм с параметром axis=1, чтобы добавить их справа-слева
    mv_table = pd.concat([mis_val, mis_val_percent], axis=1)  
    #Переименование колонок
    mv_table = mv_table.rename(columns = {0 : 'Пропущенные значения', 1 : '% от всех значений'})
    #Создаем колонку с типом данных 
    mv_table['Тип данных'] = df.dtypes
    mv_table = mv_table[mv_table.iloc[:,1]!=0].sort_values(by='Пропущенные значения',ascending=False)
    print ("Ваш датафрейм содержит " + str(df.shape[1]) + " колонок и " + str(df.shape[0]) + " строк.\n")
    print("Он имеет  " + str(mv_table.shape[0]) +" колонок с пропущенными значениями.")    
    return display(mv_table) 

## Получение данных <a class="anchor" id="id_2"></a> 

### Создание списка нужных файлов <a class="anchor" id="id_2a"></a> 

In [82]:
list_of_folders = [units_folder, boron_dlc_folder, avarice_dlc_folder, terran_dlc_folder, split_dlc_folder]
list_of_files = [] # cписок имен файлов XML с путями
list_of_XMLfiles = [] #cписок имен файлов
for folder in list_of_folders:
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".xml"):
                list_of_XMLfiles.append( file)
                #print(os.path.join(root, file))
                list_of_files.append(os.path.join(root, file))
print('Список каталогов в units_folder: ', os.listdir(units_folder))

Список каталогов в units_folder:  ['size_l', 'size_m', 'size_s', 'size_xl', 'size_xs', 'xref_parts']


In [83]:
#Удаление ошибочных файлов
skip_files = [r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units\size_l\ship_bor_l_destroyer_01_macro.xml', 
r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units\size_l\ship_bor_l_miner_solid_01_macro.xml',
r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units\size_l\ship_bor_l_trans_container_01_macro.xml',
r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units\size_s\macros\ship_bor_s_miner_solid_01_story_macro.xml',
r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units\size_xl\ship_bor_xl_carrier_01_macro.xml',
r'H:\Steam\X4\Unpacked\extensions\boron_dlc\assets\units\size_l\ship_bor_l_miner_solid_01_macro.xml']
for file in list_of_files:
    if file in skip_files:
        list_of_files.remove(file)
print(len(list_of_files))

725


### Нахождение информации в разных тегах XML на примере аргонского дестроера <a class="anchor" id="id_2b"></a> 

In [84]:
# Вывод файлов, относящихся к этому кораблю
keyword = 'ship_arg_l_destroyer_01'
arg_destroyer_files = []
for file in list_of_files :
    if keyword in file:
        print(file)
        arg_destroyer_files.append(file)
    

H:\Steam\X4\Unpacked\assets\units\size_l\ship_arg_l_destroyer_01.xml
H:\Steam\X4\Unpacked\assets\units\size_l\macros\ship_arg_l_destroyer_01_a_macro.xml
H:\Steam\X4\Unpacked\assets\units\size_l\macros\ship_arg_l_destroyer_01_b_macro.xml


In [85]:
with open(arg_destroyer_files[1], 'r') as user_file:
    file_contents = user_file.read()
#Создание древовидной структуры файла      
arg_destroyer_01a_soup = BeautifulSoup(file_contents, features="xml")
#Поиск тега connection  с атрибутом c определенным ref и в этом теге поиск тега macro 
tags = arg_destroyer_01a_soup.find('connection', attrs = {'ref':'con_storage01'}).find('macro')
tags
    

<macro connection="ShipConnection" ref="storage_arg_l_destroyer_01_a_macro"/>

Один из способов собрать информацию о теге и его атрибутах из объекта BeatifulSoup:

In [86]:
dict_with_stuff = {}
dict_with_stuff['hull_max'] = arg_destroyer_01a_soup.select_one('hull').get('max')
print(dict_with_stuff)

{'hull_max': '93000'}


Нужные теги: 
```
<explosiondamage shield="5000" value="1000"/>
<storage missile="160" unit="10"/>
<hull max="93000"/>
<secrecy level="2"/>
<purpose primary="fight"/>
<people capacity="44"/>
<shipdetail ref="shipdetail_ship_l_01"/>
<physics mass="196.016">
<inertia pitch="96.271" roll="77.016" yaw="96.271"/>
<drag forward="99.004" horizontal="73.005" pitch="106.203" reverse="396.016" roll="106.203" vertical="73.005" yaw="106.203"/>
</physics>
<thruster tags="large"/>
<ship type="destroyer"/>
</properties> 
``` 
- Оборудование 
```
<software>
<software compatible="1" ware="software_dockmk2"/>
<software default="1" ware="software_flightassistmk1"/>
<software default="1" ware="software_scannerlongrangemk1"/>
<software compatible="1" ware="software_scannerlongrangemk2"/>
<software default="1" ware="software_scannerobjectmk1"/>
<software compatible="1" ware="software_scannerobjectmk2"/>
<software default="1" ware="software_targetmk1"/>
<software compatible="1" ware="software_trademk1"/>
</software> 
```
```
      
<connection ref="con_storage01">  # ссылка на файл, содержащий значение емкости грузового отсека
    <macro ref="storage_arg_l_destroyer_01_a_macro" connection="ShipConnection" />
</connection>   
    ```

Теги, необходимые для включения в таблицу:

In [87]:
needed_tags = ['macro', 'explosiondamage', 'storage', 'hull', 'secrecy', 'purpose', 'people', 'physics', 'inertia', 'drag', 'ship']

In [88]:
list_of_tags_in_macroFiles = [] # Список для сохранения всех тегов из файла xml
for tag in arg_destroyer_01a_soup.findAll():
    #print(tag.name)
    list_of_tags_in_macroFiles.append(tag)
#print(list_of_tags_in_macroFiles)

In [89]:
#получение имени xml-файла из объекта BS
macro_tag = arg_destroyer_01a_soup.find('macro')
filename = macro_tag['name']+'.xml'
print(filename)

ship_arg_l_destroyer_01_a_macro.xml


Получение ссылки на компонент из xml- файла. Компонент ссылается на модель корабля и с его помощью можно найти ссылки на конфигурацию корабля(количество щитов. пушек. двигателей):

In [90]:
component = arg_destroyer_01a_soup.find('component')
component = component['ref']
component

'ship_arg_l_destroyer_01'

Нахождение тега типа корабля(авианосец. истребитель, торговый корабль итд):

In [91]:
ship_type = arg_destroyer_01a_soup.find('ship').attrs
print(ship_type['type'])    

destroyer


Нахождение информации о значении прочности корабля:

In [92]:
hull_max_size = arg_destroyer_01a_soup.find('hull')
print(hull_max_size)

<hull max="93000"/>


Нахождение значений аттрибутов внутри тега:

In [93]:
# Первый способ
for tag in  arg_destroyer_01a_soup.findAll('storage'):
    print(tag['missile'])
    print(tag['unit'])
#Второй способ 
print(arg_destroyer_01a_soup.findAll('storage')[0]['missile'])
print(arg_destroyer_01a_soup.findAll('storage')[0]['unit'])


160
10
160
10


Выбор только нужных тегов и включение их в список:

In [94]:
tag_values_list = []
for tag in needed_tags:
    #if tag != 'macro':
        tag_values = arg_destroyer_01a_soup.find(tag).attrs.values()
        tag_values_list.append(list(tag_values))
    #else:
        #filename = arg_destroyer_01a_soup.find('macro')['name']+'.xml'
        #tag_values_list.append(filename)
print(tag_values_list)

[['ship_arg_l_destroyer_01_a_macro', 'ship_l'], ['1000', '5000'], ['160', '10'], ['93000'], ['2'], ['fight'], ['44'], ['196.016'], ['96.271', '96.271', '77.016'], ['99.004', '396.016', '73.005', '73.005', '106.203', '106.203', '106.203'], ['destroyer']]


Вывод информации о ключах и значениях словаря атрибутов тега:

In [95]:
print(type(arg_destroyer_01a_soup.findAll('storage')))
storage = arg_destroyer_01a_soup.find('storage')
print(type(storage))
print(type(storage.attrs))
#find attribute names
print(storage.attrs.keys())
#find attribute values
print(storage.attrs.values())


<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>
<class 'dict'>
dict_keys(['missile', 'unit'])
dict_values(['160', '10'])


Результат findAll - тип ResultSet    
Результат find() - тип Tag  
Результат tag.attrs - тип dict   

Создание словаря тегов и его атрибутов:

In [96]:
#Список тегов, которые нужно собрать в качестве ключей словаря
tag_names_list = []
for tag in arg_destroyer_01a_soup.findAll(): 
    if tag.name =='macro' and 'name' in tag.attrs: # чтобы отбросить ненужные теги macro
         D = {}
         D[tag.name] = list(tag.attrs.keys()) #Создаем словарь с ключом по имени тега и его значением в виде списка атрибутов
         tag_names_list.append(D)     #Добавляем словарь в список    
    else: 
         # Если количество атрибутов в нужном теге больше 1, то добавляем в список словарь        
         if tag.name in needed_tags and len(tag.attrs) > 1 and tag.name !='macro':
              D = {}
              D[tag.name] = list(tag.attrs.keys())
              tag_names_list.append(D)
          # Если атрибут только один, то добавляем просто его имя
         elif tag.name in needed_tags and len(tag.attrs) == 1 and tag.name !='macro':                
               tag_names_list.append(tag.name)           

print(tag_names_list)
        

[{'macro': ['name', 'class']}, {'explosiondamage': ['value', 'shield']}, {'storage': ['missile', 'unit']}, 'hull', 'secrecy', 'purpose', 'people', 'physics', {'inertia': ['pitch', 'yaw', 'roll']}, {'drag': ['forward', 'reverse', 'horizontal', 'vertical', 'pitch', 'yaw', 'roll']}, 'ship']


Преобразование значений ключей словарей в списке tag_names_list на ихзначения в тегах:

In [97]:
temp = []
#count=0
for count in range(len(tag_names_list)):   
   if isinstance(tag_names_list[count], dict):       
       for key in tag_names_list[count]:
           D = {}
           D[key] = tag_values_list[count]
           temp.append(D)
   else:   
    
        temp.append(tag_values_list[count])          
print(len(temp))
print(len(tag_names_list))
tag_values_dict = temp
print(tag_values_dict)

11
11
[{'macro': ['ship_arg_l_destroyer_01_a_macro', 'ship_l']}, {'explosiondamage': ['1000', '5000']}, {'storage': ['160', '10']}, ['93000'], ['2'], ['fight'], ['44'], ['196.016'], {'inertia': ['96.271', '96.271', '77.016']}, {'drag': ['99.004', '396.016', '73.005', '73.005', '106.203', '106.203', '106.203']}, ['destroyer']]


Слияние двух списков словарей в один сложный словарь, где ключ - имя тега, его значение словарь, в котором ключами являются имена атрибутов этого тега, их значениями - значения атрибутов:

In [98]:
#Список параметров корабля
tag_names_dict = []
ship_data = {}
for tag in arg_destroyer_01a_soup.findAll():    
    if tag.name in needed_tags and not'ref' in tag.attrs:
        #print(tag.name, tag.attrs)
        D = dict(zip(tag.name, tag.attrs))
        ship_data[tag.name] = tag.attrs
print(ship_data)

        

{'macro': {'name': 'ship_arg_l_destroyer_01_a_macro', 'class': 'ship_l'}, 'explosiondamage': {'value': '1000', 'shield': '5000'}, 'storage': {'missile': '160', 'unit': '10'}, 'hull': {'max': '93000'}, 'secrecy': {'level': '2'}, 'purpose': {'primary': 'fight'}, 'people': {'capacity': '44'}, 'physics': {'mass': '196.016'}, 'inertia': {'pitch': '96.271', 'yaw': '96.271', 'roll': '77.016'}, 'drag': {'forward': '99.004', 'reverse': '396.016', 'horizontal': '73.005', 'vertical': '73.005', 'pitch': '106.203', 'yaw': '106.203', 'roll': '106.203'}, 'ship': {'type': 'destroyer'}}


### # Нахождение значений page_id и ship_name id:

In [99]:
# Нахождение значений page_id и ship_name id:
for tag in arg_destroyer_01a_soup.findAll('identification'):
    name_string = tag.attrs['name']
    page_id = re.findall('\d+',name_string)[0]
    name_id = re.findall('\d+',name_string)[1]
        #list_of_tags_in_macroFiles.append(tag)
print('ID атрибута page {0}, ID атрибута названия корабля {1}'.format(page_id, name_id))

temp_dict = {}
for file in arg_destroyer_files:
    with open(file, 'r') as user_file:
        file_contents = user_file.read()
        #Создание древовидной структуры файла      
        file_BStree = BeautifulSoup(file_contents, features="xml")
        for tag in file_BStree.findAll('identification'):
            name_string = tag.attrs['name']
            page_id = re.findall('\d+',name_string)[0]
            name_id = re.findall('\d+',name_string)[1]
            temp_dict['pageID'] = page_id
            temp_dict['nameID'] = name_id
print(temp_dict)

ID атрибута page 20101, ID атрибута названия корабля 11002
{'pageID': '20101', 'nameID': '11003'}


Варианты колонок для таблицы:

In [100]:
columns1 = pd.MultiIndex.from_arrays([['explosiondamage','explosiondamage', 'storage', 'storage', 'hull','secrecy', 'purpose','people','physics',
                                      'inertia', 'inertia','inertia', 'drag','drag','drag','drag','drag','drag','drag'],
                                     ['value','shield','missile','unit','hull','secrecy','purpose','people','physics',
                                      'pitch','yaw','roll','forward','reverse','horizontal', 'vertical', 'pitch', 'yaw', 'roll']], names=['group', 'stat'])

example1 = pd.DataFrame(columns = columns1)
display(example1)

Empty DataFrame
Columns: [(explosiondamage, value), (explosiondamage, shield), (storage, missile), (storage, unit), (hull, hull), (secrecy, secrecy), (purpose, purpose), (people, people), (physics, physics), (inertia, pitch), (inertia, yaw), (inertia, roll), (drag, forward), (drag, reverse), (drag, horizontal), (drag, vertical), (drag, pitch), (drag, yaw), (drag, roll)]
Index: []

In [101]:
columns2 = pd.MultiIndex.from_arrays([['explosiondamage','explosiondamage', 'storage', 'storage', '','', '','','',
                                      'inertia', 'inertia','inertia', 'drag','drag','drag','drag','drag','drag','drag','type'],
                                     ['value','shield','missile','unit','hull','secrecy','purpose','people','physics',
                                      'pitch','yaw','roll','forward','reverse','horizontal', 'vertical', 'pitch', 'yaw', 'roll','']], names=['group', 'stat'])
example2 = pd.DataFrame(columns = columns2)
display(example2)

Empty DataFrame
Columns: [(explosiondamage, value), (explosiondamage, shield), (storage, missile), (storage, unit), (, hull), (, secrecy), (, purpose), (, people), (, physics), (inertia, pitch), (inertia, yaw), (inertia, roll), (drag, forward), (drag, reverse), (drag, horizontal), (drag, vertical), (drag, pitch), (drag, yaw), (drag, roll), (type, )]
Index: []

In [102]:
columns3 = pd.MultiIndex.from_arrays([['explosiondamage','explosiondamage', 'storage', 'storage', 'hull','secrecy', 'purpose','people','physics',
                                      'inertia', 'inertia','inertia', 'drag','drag','drag','drag','drag','drag','drag','ship','meta','meta'],
                                     ['value','shield','missile','unit','','','','','',
                                      'pitch','yaw','roll','forward','reverse','horizontal', 'vertical', 'pitch', 'yaw', 'roll','type','filename','class']], names=['group', 'stat'])
example3 = pd.DataFrame(columns = columns3)
display(example3)

Empty DataFrame
Columns: [(explosiondamage, value), (explosiondamage, shield), (storage, missile), (storage, unit), (hull, ), (secrecy, ), (purpose, ), (people, ), (physics, ), (inertia, pitch), (inertia, yaw), (inertia, roll), (drag, forward), (drag, reverse), (drag, horizontal), (drag, vertical), (drag, pitch), (drag, yaw), (drag, roll), (ship, type), (meta, filename), (meta, class)]
Index: []

### Создание списка путей к конфигурационным файлам кораблей <a class="anchor" id="id_2b_1"></a>

In [103]:
list_of_ships = [] # список файлов с данными кораблей
list_of_ships_configs = [] #список файлов с данными по геометрии кораблей, нужен для подсчета слотов оборудования
keywords = ['ship','macro','spacesuit','cockpit','storage', 'thruster']
for file in list_of_files:
    if keywords[0] in file and keywords[1] in file and keywords[2] not in file:
        list_of_ships.append(file)
    elif (
        keywords[0] in file and keywords[1] not in file and keywords[3] not in file and keywords[4] not in file 
                and keywords[5] not in file 
    ):
        list_of_ships_configs.append(file)

print(len(list_of_ships))     
print(len(list_of_ships_configs))  

321
260


### Создание структуры тегов файла descriptions_eng <a class="anchor" id="id_2c"></a>

```<macros>
  <macro name="ship_tel_xl_builder_01_a_macro" class="ship_xl">
    <component ref="ship_arg_xl_builder_01" />
    <properties>
      <identification name="{20101,21102}" basename="{20101,21101}" description="{20101,21112}" variation="{20111,1101}" shortvariation="{20111,1103}" icon="ship_xl_build_01" />      
      <explosiondamage value="1200" shield="6000" />
      <storage missile="40" unit="100" />
      <hull max="115000" />
      <secrecy level="1" />
      <purpose primary="build" />
      <people capacity="219" />
      <physics mass="984.137">
        <inertia pitch="612.235" yaw="612.235" roll="489.788" />
        <drag forward="276.827" reverse="1107.31" horizontal="519.018" vertical="519.018" pitch="896.827" yaw="896.827" roll="896.827" />
      </physics>```

0001-l044.xml - словарь имен и описаний на английском языке.
Например, раздел  
```<page id="20101" title="Ships" descr="Names and descriptions of ships" voice="yes"> ```  
    отвечает за названия кораблей по типу {pade_id, ship_id}. Например, корабль Albatros Vanguard имеет имя в виде name="{20101,21102}:

```<identification name="{20101,21102}" basename="{20101,21101}" description="{20101,21112}" variation="{20111,1101}" shortvariation="{20111,1103}" icon="ship_xl_build_01" />```   

Имя в файле с текстом:  
```<t id="21101">Albatross</t>```</br>
```<t id="21102">(Albatross Vanguard){20101,21101} {20111,1101}</t>```

In [104]:
with open(descriptions_eng, 'r') as user_file:
    file_contents = user_file.read()
#Создание древовидной структуры файла      
descriptions_Sobject = BeautifulSoup(file_contents, features="xml")

list_of_tags_in_textfile = []
for tag in descriptions_Sobject.findAll('page', attrs = {'id':20101,'title':'Ships','descr':'Names and descriptions of ships'}):
    #print(tag.name)
    list_of_tags_in_textfile.append(tag)
    
#print(list_of_tags_in_textfile)

### Создание общей таблицы характеристик кораблей по всем файлам <a class="anchor" id="id_2d"></a>

#### Создание таблицы <a class="anchor" id="id_2d_1"></a>

In [105]:
## 
temp = [] # Временный список, куда будут присоединяться отдельные списки для каждого корабля
text_ids_list = [] # сgисок словарей значений page_id и name_id
files = [] # лист с именами файлов
files_with_ident = [] # лист с именами файлов с аттрибутом identification
print(len(list_of_ships))
for file in list_of_ships:
     if problemic_file in file:
         continue
     else:
               
        ship_data = {} 
        with open(file, 'r') as user_file:
            file_contents = user_file.read()
        #Создание древовидной структуры файла      
            file_BStree = BeautifulSoup(file_contents, features="xml")
        
       
            for tag in file_BStree.findAll():
            
                if tag.name in needed_tags and not'ref' in tag.attrs:
                #print(tag.name, tag.attrs)
                #D = dict(zip(tag.name, tag.attrs))
                    ship_data[tag.name] = tag.attrs 
                ## Create list of names
            ship_data['component'] = file_BStree.select_one('component').get('ref')
            temp.append(ship_data) 
            files.append(file)
            for tag in file_BStree.findAll('identification'):                    
                #print(tag)    
                    D = {}
                    name_string = tag.attrs['name']
                    page_id = re.findall('\d+',name_string)[0]
                    name_id = re.findall('\d+',name_string)[1]
                    D['pageID'] = page_id
                    D['nameID'] = name_id
                    text_ids_list.append(D)
                    files_with_ident.append(file)
        
                
            

## Проверка на равенство полученных списков:
if len(temp) == len(text_ids_list):
    print('Вроде хорошо')
else:
    print('Длины списков не равны')  
    print(len(temp), len(text_ids_list))
#print(text_ids_list)

321
Вроде хорошо


In [106]:
columns4 = pd.MultiIndex.from_arrays([['macro',"macro", 'explosiondamage','explosiondamage', 'storage', 'storage', 'hull','secrecy', 'purpose','people','physics',
                                      'inertia', 'inertia','inertia', 'drag','drag','drag','drag','drag','drag','drag','ship'],
                                     ["name", "class ",  'value','shield','missile','unit','','','','','',
                                      'pitch','yaw','roll','forward','reverse','horizontal', 'vertical', 'pitch', 'yaw', 'roll','type']], names=['group', 'stat'])
example4 = pd.DataFrame(columns = columns4)
display(example4)

Empty DataFrame
Columns: [(macro, name), (macro, class ), (explosiondamage, value), (explosiondamage, shield), (storage, missile), (storage, unit), (hull, ), (secrecy, ), (purpose, ), (people, ), (physics, ), (inertia, pitch), (inertia, yaw), (inertia, roll), (drag, forward), (drag, reverse), (drag, horizontal), (drag, vertical), (drag, pitch), (drag, yaw), (drag, roll), (ship, type)]
Index: []

In [107]:
ship_names = []
names_ids_list =  []
for element in text_ids_list:    
    ex = descriptions_Sobject.find('page', attrs = {'id':20101, 'title':'Ships'}).find('t', attrs = {'id':element['nameID']})
    try: 
        name = re.search('\D+',ex.text).group()
        name = re.split('[^A-Za-z]',name)
        name = list(filter(None, name))
        new_name = " ".join([ele for ele in name if  ele[0].isupper()])
    except AttributeError:
        continue
    finally:
        ship_names.append(new_name)
        names_ids_list.append(element['nameID'])
    
print(ship_names)   

['Behemoth Vanguard', 'Behemoth Sentinel', 'Magnetar Gas Vanguard', 'Magnetar Gas Sentinel', 'Magnetar Mineral Vanguard', 'Magnetar Mineral Sentinel', 'Veles Vanguard', 'Veles Sentinel', 'Mokosi Vanguard', 'Mokosi Sentinel', 'Incarcatura Vanguard', 'Incarcatura Sentinel', 'Shuyaku Vanguard', 'Shuyaku Sentinel', 'Sonra Vanguard', 'Sonra Sentinel', 'Odysseus Vanguard', 'Odysseus Sentinel', 'Odysseus E', 'Chthonios Gas Vanguard', 'Chthonios Gas Sentinel', 'Chthonios E Gas', 'Chthonios Mineral Vanguard', 'Chthonios Mineral Sentinel', 'Chthonios E Mineral', 'Helios Vanguard', 'Helios Sentinel', 'Selene Vanguard', 'Selene Sentinel', 'Helios E', 'Phoenix Vanguard', 'Phoenix Sentinel', 'Crane Gas Vanguard', 'Crane Gas Sentinel', 'Crane Mineral Vanguard', 'Crane Mineral Sentinel', 'Pelican Vanguard', 'Pelican Sentinel', 'Heron Vanguard', 'Heron Sentinel', 'Minotaur Vanguard', 'Minotaur Sentinel', 'Minotaur Raider', 'Cerberus Vanguard', 'Cerberus Sentinel', 'Sunder Gas Vanguard', 'Sunder Gas Sen

In [108]:
## Creating table    
ship_data = pd.DataFrame.from_dict(temp)
# Выбор стиля названий колонок
#ship_data.columns = columns4 
display(ship_data.head(2))

,macro,explosiondamage,storage,hull,secrecy,purpose,people,physics,inertia,drag,ship,component
0,"{'name': 'ship_arg_l_destroyer_01_a_macro', 'c...","{'value': '1000', 'shield': '5000'}","{'missile': '160', 'unit': '10'}",{'max': '93000'},{'level': '2'},{'primary': 'fight'},{'capacity': '44'},{'mass': '196.016'},"{'pitch': '96.271', 'yaw': '96.271', 'roll': '...","{'forward': '99.004', 'reverse': '396.016', 'h...",{'type': 'destroyer'},ship_arg_l_destroyer_01
1,"{'name': 'ship_arg_l_destroyer_01_b_macro', 'c...","{'value': '1000', 'shield': '5000'}","{'missile': '160', 'unit': '10'}",{'max': '111000'},{'level': '2'},{'primary': 'fight'},{'capacity': '36'},{'mass': '235.22'},"{'pitch': '103.378', 'yaw': '103.378', 'roll':...","{'forward': '108.805', 'reverse': '435.22', 'h...",{'type': 'destroyer'},ship_arg_l_destroyer_01


In [109]:
component = ship_data['component']

In [110]:
columns_to_expand = ['macro', 'explosiondamage',	'storage',	'hull',	 'secrecy',	'purpose',	'people',	'physics',	'inertia',	'drag','ship']
for column in columns_to_expand:
    ship_data = pd.concat([ship_data.drop([column], axis = 1), ship_data[column].apply(pd.Series)], axis = 1)   

In [111]:
ship_data = ship_data.drop(['alias',0,'time','countermeasure'],axis = 1)
ship_data['component'] = component
display(ship_data.head(2))

,component,name,class,value,shield,missile,unit,max,level,primary,capacity,mass,pitch,yaw,roll,forward,reverse,horizontal,vertical,pitch,yaw,roll,type
0,ship_arg_l_destroyer_01,ship_arg_l_destroyer_01_a_macro,ship_l,1000,5000,160,10,93000,2,fight,44,196.016,96.271,96.271,77.016,99.004,396.016,73.005,73.005,106.203,106.203,106.203,destroyer
1,ship_arg_l_destroyer_01,ship_arg_l_destroyer_01_b_macro,ship_l,1000,5000,160,10,111000,2,fight,36,235.22,103.378,103.378,82.702,108.805,435.22,87.605,87.605,114.044,114.044,114.044,destroyer


Переименование колонок

In [112]:
column_names = ['component','filename','ship_class','explosiondamage_shield',
                'explosiondamage_value','storage_missile','storage_unit','hull',
                'secrecy_level','purpose','people_capacity','ship_mass',
                'inertia_pitch', 'inertia_roll', 'inertia_yaw',
                'drag_forward', 'drag_reverse', 'drag_horizontal','drag_vertical',
                'drag_pitch','drag_yaw', 'drag_roll', 'ship_type']
ship_data.columns = column_names

Добавление колонки с именем корабля

In [113]:
# Проверка на равенство длин:
if len(ship_data) == len(ship_names):
    ship_data['name'] = ship_names
    #Do some rearranging of columns' order
    cols = ship_data.columns.to_list()
    cols = cols[-1:] + cols[:-1]
    ship_data_new = ship_data[cols]
else:
    print('Длины не равны')

Добавление ID имени корабля

In [114]:
ship_data['name_id'] = names_ids_list

Вывод первых пяти строк полученной таблицы и информации о ней:

In [115]:
display(ship_data.head())
print(ship_data.info())

,component,filename,ship_class,explosiondamage_shield,explosiondamage_value,storage_missile,storage_unit,hull,secrecy_level,purpose,people_capacity,ship_mass,inertia_pitch,inertia_roll,inertia_yaw,drag_forward,drag_reverse,drag_horizontal,drag_vertical,drag_pitch,drag_yaw,drag_roll,ship_type,name,name_id
0,ship_arg_l_destroyer_01,ship_arg_l_destroyer_01_a_macro,ship_l,1000,5000,160,10,93000,2,fight,44,196.016,96.271,96.271,77.016,99.004,396.016,73.005,73.005,106.203,106.203,106.203,destroyer,Behemoth Vanguard,11002
1,ship_arg_l_destroyer_01,ship_arg_l_destroyer_01_b_macro,ship_l,1000,5000,160,10,111000,2,fight,36,235.22,103.378,103.378,82.702,108.805,435.22,87.605,87.605,114.044,114.044,114.044,destroyer,Behemoth Sentinel,11003
2,ship_arg_l_miner_liquid_01,ship_arg_l_miner_liquid_01_a_macro,ship_l,800,4000,30,10,26000,1,mine,46,205.27,133.749,133.749,106.999,56.738,324.216,126.666,126.666,140.897,140.897,140.897,largeminer,Magnetar Gas Vanguard,11104
3,ship_arg_l_miner_liquid_01,ship_arg_l_miner_liquid_01_b_macro,ship_l,800,4000,30,10,32000,1,mine,38,246.324,147.778,147.778,118.223,62.485,357.059,151.999,151.999,155.677,155.677,155.677,largeminer,Magnetar Gas Sentinel,11105
4,ship_arg_l_miner_solid_01,ship_arg_l_miner_solid_01_a_macro,ship_l,800,4000,30,10,26000,1,mine,46,204.245,132.733,132.733,106.186,56.594,323.396,127.239,127.239,140.528,140.528,140.528,largeminer,Magnetar Mineral Vanguard,11102


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   component               321 non-null    object
 1   filename                321 non-null    object
 2   ship_class              321 non-null    object
 3   explosiondamage_shield  166 non-null    object
 4   explosiondamage_value   164 non-null    object
 5   storage_missile         232 non-null    object
 6   storage_unit            106 non-null    object
 7   hull                    321 non-null    object
 8   secrecy_level           318 non-null    object
 9   purpose                 321 non-null    object
 10  people_capacity         243 non-null    object
 11  ship_mass               321 non-null    object
 12  inertia_pitch           320 non-null    object
 13  inertia_roll            320 non-null    object
 14  inertia_yaw             320 non-null    object
 15  drag_f

#### Обработка таблицы на наличие ошибок <a class="anchor" id="id_2d_2"></a>

Вывод информации о пропущенных значениях:

In [116]:
missing_values_tab(ship_data)

Ваш датафрейм содержит 25 колонок и 321 строк.

Он имеет  15 колонок с пропущенными значениями.


,Пропущенные значения,% от всех значений,Тип данных
storage_unit,215,66.98,object
explosiondamage_value,157,48.91,object
explosiondamage_shield,155,48.29,object
storage_missile,89,27.73,object
people_capacity,78,24.30,object
secrecy_level,3,0.93,object
inertia_pitch,1,0.31,object
inertia_roll,1,0.31,object
inertia_yaw,1,0.31,object
drag_horizontal,1,0.31,object


Изменение типа колонок. Пропуски в количественных данных заменю нулями, так как их наличие означает  отсутствие этого параметра (например ракет у кораблей майнеров).

In [117]:
#Изменение типа колонок на float
cols_to_float = ['ship_mass',	'inertia_pitch',	'inertia_yaw',	'inertia_roll',	'drag_forward',
                 'drag_reverse',	'drag_horizontal',	'drag_vertical',	'drag_pitch',	'drag_yaw',	'drag_roll'	]
try:
    for column in cols_to_float:
        ship_data[column] = ship_data[column].fillna(0)
        ship_data[column] = ship_data[column].astype('float')

except BaseException:
    print(column)
#Изменение типа колонок на int
cols_to_int = ['explosiondamage_value',	'explosiondamage_shield',	'storage_missile',
               'storage_unit',	'hull',	'secrecy_level', 'people_capacity']
try:
    for column in cols_to_int:
        ship_data[column] = ship_data[column].fillna(0)        
        ship_data[column] = ship_data[column].astype('int')
except BaseException:
    print(column)

Пропуски в категориальных данных заменю словом "unknown"

In [118]:
ship_data['purpose'] = ship_data['purpose'].fillna('unknown')
ship_data['ship_type'] = ship_data['ship_type'].fillna('unknown')

In [119]:
ship_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   component               321 non-null    object 
 1   filename                321 non-null    object 
 2   ship_class              321 non-null    object 
 3   explosiondamage_shield  321 non-null    int32  
 4   explosiondamage_value   321 non-null    int32  
 5   storage_missile         321 non-null    int32  
 6   storage_unit            321 non-null    int32  
 7   hull                    321 non-null    int32  
 8   secrecy_level           321 non-null    int32  
 9   purpose                 321 non-null    object 
 10  people_capacity         321 non-null    int32  
 11  ship_mass               321 non-null    float64
 12  inertia_pitch           321 non-null    float64
 13  inertia_roll            321 non-null    float64
 14  inertia_yaw             321 non-null    fl

### Справочный раздел <a class="anchor" id="id_2e"></a>

In [120]:
##find name
#ex = descriptions_Sobject.find('t', attrs = {'id':1})
ex = descriptions_Sobject.find('page', attrs = {'id':20101, 'title':'Ships'}).find('t', attrs = {'id':10412})
#type(ex.contents)
name = re.search('\D+',ex.text).group()
name = re.split('[^A-Za-z]',name)
name = list(filter(None, name))
new_name = " ".join([ele for ele in name if  ele[0].isupper()])
new_name

''

In [121]:
##find name
#ex = descriptions_Sobject.find('t', attrs = {'id':1})
ex = descriptions_Sobject.find('page', attrs = {'id':20111}).find('t',attrs = {'id':1203})
#type(ex.contents)
name = re.search('\D+',ex.text).group()
name

'ST'

### Нахождение информации о количестве слотов  на кораблях  <a class="anchor" id="id_2f"></a>

Информация о количестве слотов щитов находится в каталоге с кораблями, например:  
h:/Steam/X4/Unpacked/assets/units/size_xl/ship_arg_xl_carrier_01.xml  
```  
 <connection name="con_shieldgen_xl_01" tags="extralarge shield standard ">
				<offset>
					<position x="179.2407" y="140.9466" z="-461.2684"/>
				</offset>
			</connection>  
```

In [122]:
with open(r'h:/Steam/X4/Unpacked/assets/units/size_xl/ship_arg_xl_carrier_01.xml', 'r') as user_file:
    file_contents = user_file.read()
#Создание древовидной структуры файла      
Sobject = BeautifulSoup(file_contents, features="xml")

In [123]:
tags = Sobject.findAll('connection', attrs = {'tags':"extralarge shield standard "})
print("Количество слотов под XL щиты: ",len(tags))

Количество слотов под XL щиты:  3


Подсчет количества турелей на M кораблях:

In [124]:
tags="turret medium standard platformcollision unhittable  combat "
tags_e = "engine medium  standard "
# Таг щитов (не на турелях)
tags_s="medium shield unhittable platformcollision standard "
with open(r'h:\Steam\X4\Unpacked\assets\units\size_m\ship_arg_m_frigate_01.xml', 'r') as user_file:
    file_contents = user_file.read()
#Создание древовидной структуры файла      
Sobject = BeautifulSoup(file_contents, features="xml")
t = Sobject.findAll('connection', attrs = {'tags':tags})
e = Sobject.findAll('connection', attrs = {'tags':tags_e})
s = Sobject.findAll('connection', attrs = {'tags':tags_s})
print("Количество слотов под турели: ",len(t))
print("Количество слотов под двигатели: ",len(e))
print("Количество слотов под щиты: ",len(s))

Количество слотов под турели:  4
Количество слотов под двигатели:  2
Количество слотов под щиты:  3


количество двигателей на S кораблях:

In [125]:
tag = "engine small platformcollision  standard "
with open(r'h:\Steam\X4\Unpacked\assets\units\size_s\ship_arg_s_fighter_01.xml', 'r') as user_file:
    file_contents = user_file.read()
#Создание древовидной структуры файла      
Sobject = BeautifulSoup(file_contents, features="xml")
t = Sobject.findAll('connection', attrs = {'tags':tag})
print(len(t))

2


Алгоритм:  
1. Заход в ```units_folder```
2. Обход папок с кораблями(кроме папки size_xs и xref_parts).
3. В каждой папке на верхнем уровне ищем xml-файлы со словом ship в названии.
4. Создаем Sobject для каждого файла с считаем количество тегов с щитами (если название папки size_xl, то ищем теги с атрибутом tags = "extralarge shield standard ")

Конфигурационные файлы кораблей связаны с файлами с их геометрией посредством ссылки на компонент, например:  
файл с геометрией имеет таг   
```<component name="ship_arg_l_destroyer_01" class="ship_l">```   
файл с конфигурацией:  
```<macros>
  <macro name="ship_arg_l_destroyer_01_a_macro" class="ship_l">
    <component ref="ship_arg_l_destroyer_01" />```     
По этому общему компоненту можно связать эти два файла и добавить данные в общую таблицу

Cписки тегов оборудования в xml-файлах кораблей:

In [126]:
# Корабли боронов
boron_shield_tags = ["boron extralarge shield", # щит на XL кораблях
                     "boron large shield",  # щит на L кораблях
                     "boron hittable medium shield", # средние щиты на турелях L/XL кораблей
                     "boron medium shield unhittable", #  щит на M кораблях  
                     "boron shield small unhittable" # щит на S  кораблях
                     ]
boron_turret_tags = ["boron combat large missile turret",   #  L турели                
                     "boron combat hittable medium missile turret", # М турели
                     "boron combat medium turret unhittable",  # М турели на М кораблях
                     "boron large mining turret" # L турели на кораблях-майнерах руды
                     ]
boron_engine_tags = ["boron engine extralarge", "boron engine large", "boron engine medium", "boron engine small"]
# Корабли остальных фракций
shield_tags = ["extralarge shield standard", # XL щиты
                "large shield  standard",  # L щиты
                "medium shield hittable  standard", #средние щиты на турелях L/XL кораблей
                "medium shield unhittable platformcollision standard", # М щит на M кораблях
                "small shield unhittable  standard" # S щит  
                ]
turret_tags = ["turret large standard missile  combat", #L турели на  XL/L кораблях
               "turret medium standard missile hittable  combat", # M турели на  XL/L кораблях 
                "turret medium standard platformcollision unhittable  combat" # М турели на  M кораблях
                "turret large mining standard" # L турели на кораблях-майнерах руды 
               ]
engine_tags = ["engine extralarge standard", # XL двигатели
               "engine large  standard",  # L двигатели
                 "engine medium  standard",  # M двигатели
                 "engine small platformcollision  standard" # S двигатели
                 ]

In [127]:
ship_config = {} # Словарь, где ключом выступает компонент корабля, его значениями - словарь с параметрами конфигурации
for file in list_of_ships_configs:
    with open(file, 'r') as user_file:
        file_contents = user_file.read()
        #Создание древовидной структуры файла      
        file_Sobject = BeautifulSoup(file_contents, features="xml")
        

        file_data = {} # Словарь для сохранения конфигурации корабля
        # Получение имени компонента
        component =  file_Sobject.select_one('component').get('name').strip()
        file_data['class'] = file_Sobject.select_one('component').get('class')
        # подсчет количества щитов
        try:
           # 
            if  file_data['class'] =='ship_xl':
               #подсчет XL щитов
               tags_xls = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*extralarge)(?=.*shield)')})
               file_data['primary_shield_slots'] = len(tags_xls)
               #подсчет M щитов
               tags_xms = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*medium)(?=.*shield)(?=.*hittable)')})
               file_data['turret_shield_slots'] = len(tags_xms)
               # подсчет L турелей               
               tags_xlt = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*large)(?=.*turret)')})
               file_data['l_turret_slots'] = len(tags_xlt)
               #подсчет М турелей
               tags_xmt = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*medium)(?=.*turret)(?=.*hittable)')})
               file_data['m_turret_slots'] = len(tags_xmt)
               #подсчет двигателей
               tags_xe = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*extralarge)(?=.*engine)')})
               file_data['engines_slots'] = len(tags_xe)
               # Сохранение словаря в главный словарь под ключом компонента
               ship_config[component] = file_data
             # В случае L корабля
            if  file_data['class'] =='ship_l':
               
               #подсчет L щитов
               tags_ls = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*large)(?=.*shield)')})
               file_data['primary_shield_slots'] = len(tags_ls)
                #подсчет  M щитов
               tags_lms = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*medium)(?=.*shield)(?=.*hittable)')})
               file_data['turret_shield_slots'] = len(tags_lms)
               # подсчет L турелей               
               tags_lt = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*large)(?=.*turret)')})
               file_data['l_turret_slots'] = len(tags_lt)
               #подсчет количества M турелей
               tags_lmt = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*medium)(?=.*turret)(?=.*hittable)')})
               file_data['m_turret_slots'] = len(tags_lmt)
               #подсчет двигателей
               tags_le = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*large)(?=.*engine)')})
               file_data['engines_slots'] = len(tags_le)
               ship_config[component] = file_data
               
               # В случае М корабля
            if  file_data['class'] =='ship_m':               
                #подсчет M щитов
               tags_ms = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*medium)(?=.*shield)(?=.*unhittable)')})
               file_data['primary_shield_slots'] = len(tags_ms) 
               # число щитов на турелях равно нулю
               file_data['turret_shield_slots'] = 0
               # число L турелей равно нулю
               file_data['l_turret_slots'] = 0          
               #подсчет М турелей
               tags_mt = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*medium)(?=.*turret)(?=.*unhittable)')})
               file_data['m_turret_slots'] = len(tags_mt)
               #подсчет двигателей
               tags_me = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*medium)(?=.*engine)')})
               file_data['engines_slots'] = len(tags_me)
               ship_config[component] = file_data
             
             # Если это S корабль
            if  file_data['class'] =='ship_s':
               # подсчет количества щитов
               tags_ss = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*small)(?=.*shield).*$')})
               file_data['primary_shield_slots'] = len(tags_ss)               
               #подсчет двигателей
               tags_se = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*small)(?=.*engine).*$')})
               file_data['engines_slots'] = len(tags_se) 
               # остальные параметры равны нулю
               file_data['turret_shield_slots'] = 0
               file_data['l_turret_slots'] = 0
               file_data['m_turret_slots'] = 0
               # сохранение словаря в главный словарь
               ship_config[component] = file_data
            # Если это корабль-нпс, полиция или дрон на станции
            if  file_data['class'] =='ship_xs':                            
               #они имеют только двигатели
               tags_se = file_Sobject.findAll('connection', attrs = {'tags':re.compile('^(?=.*small)(?=.*engine).*$')})
               file_data['engines_slots'] = len(tags_se) 
               file_data['primary_shield_slots'] = 0 
               file_data['turret_shield_slots'] = 0
               file_data['l_turret_slots'] = 0
               file_data['m_turret_slots'] = 0
               ship_config[component] = file_data

        except:           
           print('error!!!',file)
#print(ship_config)

Создание таблицы из полученного словаря словарей:

In [128]:
ship_configs_df = pd.DataFrame.from_dict(ship_config,orient = 'index').reset_index().rename(columns = {'index':'component'})
display(ship_configs_df.head())

,component,class,primary_shield_slots,turret_shield_slots,l_turret_slots,m_turret_slots,engines_slots
0,ship_arg_l_destroyer_01,ship_l,3,9,2,8,3
1,ship_arg_l_miner_liquid_01,ship_l,2,7,0,6,2
2,ship_arg_l_miner_solid_01,ship_l,2,9,1,6,2
3,ship_arg_l_trans_container_01,ship_l,2,5,0,7,2
4,ship_arg_l_trans_container_02,ship_l,2,5,0,7,2


##### Слияние таблиц ship_data и ship_configs

In [129]:
ship_data = ship_data.merge(ship_configs_df, on = 'component', how = 'left')
# Избавление от дублирующейся колонки
ship_data = ship_data.drop('class', axis =1)
#Пропуски можно заполнить нулями, так как их наличие означает  отсутствие слотов 
ship_data = ship_data.fillna(0)

In [130]:
#Изменение типа колонок на int
cols_to_int = ['primary_shield_slots',	'turret_shield_slots',	'l_turret_slots',	'm_turret_slots',	'engines_slots']
try:
    for column in cols_to_int:
        ship_data[column] = ship_data[column].fillna(0)        
        ship_data[column] = ship_data[column].astype('int')
except BaseException:
    print(column)
ship_data.head()

,component,filename,ship_class,explosiondamage_shield,explosiondamage_value,storage_missile,storage_unit,hull,secrecy_level,purpose,people_capacity,ship_mass,inertia_pitch,inertia_roll,inertia_yaw,drag_forward,drag_reverse,drag_horizontal,drag_vertical,drag_pitch,drag_yaw,drag_roll,ship_type,name,name_id,primary_shield_slots,turret_shield_slots,l_turret_slots,m_turret_slots,engines_slots
0,ship_arg_l_destroyer_01,ship_arg_l_destroyer_01_a_macro,ship_l,1000,5000,160,10,93000,2,fight,44,196.016,96.271,96.271,77.016,99.004,396.016,73.005,73.005,106.203,106.203,106.203,destroyer,Behemoth Vanguard,11002,3,9,2,8,3
1,ship_arg_l_destroyer_01,ship_arg_l_destroyer_01_b_macro,ship_l,1000,5000,160,10,111000,2,fight,36,235.220,103.378,103.378,82.702,108.805,435.220,87.605,87.605,114.044,114.044,114.044,destroyer,Behemoth Sentinel,11003,3,9,2,8,3
2,ship_arg_l_miner_liquid_01,ship_arg_l_miner_liquid_01_a_macro,ship_l,800,4000,30,10,26000,1,mine,46,205.270,133.749,133.749,106.999,56.738,324.216,126.666,126.666,140.897,140.897,140.897,largeminer,Magnetar Gas Vanguard,11104,2,7,0,6,2
3,ship_arg_l_miner_liquid_01,ship_arg_l_miner_liquid_01_b_macro,ship_l,800,4000,30,10,32000,1,mine,38,246.324,147.778,147.778,118.223,62.485,357.059,151.999,151.999,155.677,155.677,155.677,largeminer,Magnetar Gas Sentinel,11105,2,7,0,6,2
4,ship_arg_l_miner_solid_01,ship_arg_l_miner_solid_01_a_macro,ship_l,800,4000,30,10,26000,1,mine,46,204.245,132.733,132.733,106.186,56.594,323.396,127.239,127.239,140.528,140.528,140.528,largeminer,Magnetar Mineral Vanguard,11102,2,9,1,6,2


In [131]:
missing_values_tab(ship_data)

Ваш датафрейм содержит 30 колонок и 321 строк.

Он имеет  0 колонок с пропущенными значениями.


,Пропущенные значения,% от всех значений,Тип данных


## Получение информации о модулях корабля <a class="anchor" id="id_3"></a>

#### Shields <a class="anchor" id="id_3a"></a>

##### Структура файлов

In [132]:
main_catalog = r'h:/Steam/X4/Unpacked/assets/props'
# Путь к каталогу с щитами
cat_shields = r'h:\Steam\X4\Unpacked\assets\props\SurfaceElements\macros'
#dlc ships shields 
split_shields = r'h:\Steam\X4\Unpacked\extensions\split_dlc\assets\props\surfaceelements\macros'
avarice_shields = r'h:\Steam\X4\Unpacked\extensions\avarice_dlc\assets\props\surfaceelements\macros'
terran_shields = r'h:\Steam\X4\Unpacked\extensions\terran_dlc\assets\props\surfaceelements\macros'
boron_shields = r'h:\Steam\X4\Unpacked\extensions\boron_dlc\assets\props\surfaceelements\macros'

Корабль Behemot Vanguard , Id = 11002

раздел в descriptions_eng  
```<page id="20106" title="Shields" descr="Names and descriptions of ship and station shields" voice="no">```

Пример
```
<macros>
  <macro name="shield_arg_l_standard_01_mk1_macro" class="shieldgenerator">
    <component ref="shield_arg_l_standard_01_mk1" />
    <properties>
      <identification name="{20106,3004}" basename="{20106,3001}" shortname="{20106,3005}" makerrace="argon" description="{20106,3002}" mk="1" />
      <recharge max="38844" rate="173" delay="0" />
      <hull max="2000" threshold="0.2" />
    </properties>
  </macro>
</macros>```

##### Исследование на примере Argon L shield mk1

In [133]:
##find name
#ex = descriptions_Sobject.find('t', attrs = {'id':1})
ex = descriptions_Sobject.find('page', attrs = {'id':20106}).find('t',attrs = {'id':3004})
#type(ex.contents)
name = re.search('[^()]+',ex.text).group()
name

'ARG L Shield Generator Mk1'

In [134]:
with open(r'h:\Steam\X4\Unpacked\assets\props\SurfaceElements\macros\shield_arg_l_standard_01_mk1_macro.xml', 'r') as user_file:
    file_contents = user_file.read()
#Создание древовидной структуры файла      
argon_shield = BeautifulSoup(file_contents, features="xml")

In [135]:
shield_page = descriptions_Sobject.find('page', attrs = {'id':20106})
shield_data = {} # Словарь для сохранения информации о щитах
# Получение имени файла этого щита
shield_data['filename'] = argon_shield.select_one('macro').get('name')+".xml"
#получение page_id  и id из переменной descriptions_eng
temp = argon_shield.select_one('identification').get('name')
name = re.search('[^{}]+',temp).group()
name_list = name.split(',')
shield_page_id = int(name_list[0]) #page_id
shield_id = int(name_list[1])       #  id щита 
shield_data['page_id'] = shield_page_id  # сохранение  page_id в словарь
shield_data['shield_id'] = shield_id     # сохранение  id в словарь
# получение информации о производителе
shield_data['maker'] = argon_shield.select_one('identification').get('makerrace')
# версия щита
shield_data['version'] = argon_shield.select_one('identification').get('mk')
shield_data['version'] = int(shield_data['version'])
# количество очков щита
shield_data['shield_value'] = argon_shield.select_one('recharge').get('max')
shield_data['shield_value'] = int(shield_data['shield_value'])
#показатель регенерации щита
shield_data['recharge_rate'] = argon_shield.select_one('recharge').get('rate')
shield_data['recharge_rate'] = int(shield_data['recharge_rate'])
#задержка регенерации щита
shield_data['recharge_delay'] = argon_shield.select_one('recharge').get('delay')
shield_data['recharge_delay'] = int(shield_data['recharge_delay'])
# количество очков жизни щита (для кораблей L/XL)
try:

    shield_data['hull'] = argon_shield.select_one('hull').get('max')
    shield_data['hull'] = int(shield_data['hull'])
except:
    shield_data['hull'] = 0

#Словарь словарей, где ключ - имя щита, взятое из файла с текстом. Его значением будет полученный выше словарь с характеристиками щита
shield_info = {}
try:
    tmp = shield_page.find('t',attrs = {'id':shield_data['shield_id']})
    name = re.search('[^()]+',tmp.text).group()   
    shield_info[name] = shield_data
except:    
    print("Error")
    print(shield_data)
print(shield_data)
print(shield_info)

{'filename': 'shield_arg_l_standard_01_mk1_macro.xml', 'page_id': 20106, 'shield_id': 3004, 'maker': 'argon', 'version': 1, 'shield_value': 38844, 'recharge_rate': 173, 'recharge_delay': 0, 'hull': 2000}
{'ARG L Shield Generator Mk1': {'filename': 'shield_arg_l_standard_01_mk1_macro.xml', 'page_id': 20106, 'shield_id': 3004, 'maker': 'argon', 'version': 1, 'shield_value': 38844, 'recharge_rate': 173, 'recharge_delay': 0, 'hull': 2000}}


Алгоритм получения:  
1. Заходим в каталог ```cat_shields```
2. Ищем файлы c shield в названии и заносим их в список.
3. Итерация через полученный список и создание словаря с данными для каждого файла.
4. Создание таблицы из полученного словаря.

Следует учесть, что средние щиты идут в двух вариантах - как щиты на кораблях класса M и как щиты на турелях кораблей L/XL. В последнем случае они имеют показатель hull, не равный 0

##### Создание списка файлов с щитами

In [136]:
list_of_shield_files = []
# Список каталогов с данными о щитах
shield_folders = [cat_shields,split_shields,avarice_shields,boron_shields,terran_shields ]
for folder in shield_folders:
        for root, dirs, files in os.walk(folder):                
                for file in files:
                        if  file.startswith("shield"):                 
                                #print(os.path.join(root, file))
                                list_of_shield_files.append(os.path.join(root, file))
# Удаление ненужных файлов
list_of_shield_files.remove(r'h:\Steam\X4\Unpacked\extensions\avarice_dlc\assets\props\surfaceelements\macros\shield_gen_m_yacht_01_mk1_video_macro.xml')
list_of_shield_files.remove(r'h:\Steam\X4\Unpacked\extensions\boron_dlc\assets\props\surfaceelements\macros\shield_bor_s_standard_01_mk1_video_macro.xml')
list_of_shield_files.remove(r'h:\Steam\X4\Unpacked\extensions\boron_dlc\assets\props\surfaceelements\macros\shield_bor_s_standard_01_mk2_video_macro.xml')
list_of_shield_files.remove(r'h:\Steam\X4\Unpacked\extensions\boron_dlc\assets\props\surfaceelements\macros\shield_bor_s_standard_01_mk3_video_macro.xml')
print(len(list_of_shield_files))


82


In [137]:
# информация о названиях, она находится в page id = 20106 в переменной descriptions_eng
shield_page = descriptions_Sobject.find('page', attrs = {'id':20106})
shield_info = {} # словарь словарей для сбора информации
count = 0
#Проход по списку нужных файлов:
for file in list_of_shield_files:
    with open(file, 'r') as user_file:
        file_contents = user_file.read()
        #Создание древовидной структуры файла      
        shield_Sobject = BeautifulSoup(file_contents, features="xml")
        

        shield_data = {} # словарь для сохранения информации о щите
        # имя файла
        shield_data['filename'] =  shield_Sobject.select_one('macro').get('name')+".xml"
        #  page_id и id щита из descriptions_eng
        try:
            temp =  shield_Sobject.select_one('identification').get('name')
            name = re.search('[^{}]+',temp).group()
            name_list = name.split(',')
            shield_page_id = int(name_list[0]) #page_id
            shield_id = int(name_list[1])       #  id щита 
            shield_data['page_id'] = shield_page_id  # сохранение page_id в словарь
            shield_data['shield_id'] = shield_id     # сохранение id в словарь
        except:
            print('identification name error')
            print(file)
        # производитель щита
        shield_data['maker'] =  shield_Sobject.select_one('identification').get('makerrace')
        # версия щита
        try:
            shield_data['version'] =  shield_Sobject.select_one('identification').get('mk')
            shield_data['version'] = int(shield_data['version'])
        except:
            print('version error')
            print(file)
        # очки щита
        try:
            shield_data['shield_value'] =  shield_Sobject.select_one('recharge').get('max')
            shield_data['shield_value'] = int(shield_data['shield_value'])
        except:
            print('shield value error')
            print(file)
        #показатель регенерации щита
        try:
            shield_data['recharge_rate'] =  shield_Sobject.select_one('recharge').get('rate')
            shield_data['recharge_rate'] = int(shield_data['recharge_rate'])
        except:
            print('recharge rate error')
            print(file)
        #задержка регенерации щита
        try:
            shield_data['recharge_delay'] =  shield_Sobject.select_one('recharge').get('delay')
            shield_data['recharge_delay'] = float(shield_data['recharge_delay'])
        except:
            print('recharge delay error')
            print(file)
        # очки жизни щита 
        try:

            shield_data['hull'] =  shield_Sobject.select_one('hull').get('max')
            shield_data['hull'] = int(shield_data['hull'])
        except:
            shield_data['hull'] = 0
        
        count +=1
#Сохранение собранной информации в главный словарь под ключом названия щита

        try:
            tmp = shield_page.find('t',attrs = {'id':shield_data['shield_id']})
            name = re.search('[^()]+',tmp.text).group()   
            shield_info[name] = shield_data
        except:    
            print("Error")
            print(shield_data)
#print(shield_data)
#print(shield_info)
print(len(shield_info))
print( count)

65
82


##### Создание и сохранение датафрейма

In [138]:
shields_df = pd.DataFrame.from_dict(shield_info,orient = 'index').reset_index().rename(columns = {'index':'name'})
display(shields_df.head())

,name,filename,page_id,shield_id,maker,version,shield_value,recharge_rate,recharge_delay,hull
0,ARG L Shield Generator Mk1,shield_arg_l_standard_01_mk1_macro.xml,20106,3004,argon,1,38844,173,0.0,2000
1,ARG L Shield Generator Mk2,shield_arg_l_standard_01_mk2_macro.xml,20106,3044,argon,2,46282,268,0.0,2000
2,ARG M Shield Generator Mk1,shield_arg_m_standard_02_mk1_macro.xml,20106,2004,argon,1,5147,26,0.5,500
3,ARG M Shield Generator Mk2,shield_arg_m_standard_02_mk2_macro.xml,20106,2044,argon,2,6133,41,0.5,500
4,ARG S Shield Generator Mk1,shield_arg_s_standard_01_mk1_macro.xml,20106,1004,argon,1,827,82,12.1,0


In [139]:
#Show info
missing_values_tab(shields_df)
shields_df.info()


Ваш датафрейм содержит 10 колонок и 65 строк.

Он имеет  0 колонок с пропущенными значениями.


,Пропущенные значения,% от всех значений,Тип данных


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            65 non-null     object 
 1   filename        65 non-null     object 
 2   page_id         65 non-null     int64  
 3   shield_id       65 non-null     int64  
 4   maker           65 non-null     object 
 5   version         65 non-null     int64  
 6   shield_value    65 non-null     int64  
 7   recharge_rate   65 non-null     int64  
 8   recharge_delay  65 non-null     float64
 9   hull            65 non-null     int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 5.2+ KB


Отсутсвуют файлы м щитов ```shield_***_m_standard_01_mk1_macro```, которые содержат не все параметры по сравнению  со второй версией вида ```shield_***_m_standard_02_mk1_macro.xml```. Но так как основные параметры в обеих версиях одинаковы, отсутствие этих файлов не влияет на точность собранной информации.


In [140]:
# проверка на ошибки
display(shields_df.query('maker=="split"'))

,name,filename,page_id,shield_id,maker,version,shield_value,recharge_rate,recharge_delay,hull
33,SPL L Shield Generator Mk1,shield_spl_l_standard_01_mk1_macro.xml,20106,3084,split,1,33018,140,0.00,3000
34,SPL L Shield Generator Mk2,shield_spl_l_standard_01_mk2_macro.xml,20106,3094,split,2,39340,217,0.00,3000
35,SPL M Shield Generator Mk1,shield_spl_m_standard_02_mk1_macro.xml,20106,2084,split,1,4375,21,0.36,500
36,SPL M Shield Generator Mk2,shield_spl_m_standard_02_mk2_macro.xml,20106,2094,split,2,5213,33,0.36,500
37,SPL S Shield Generator Mk1,shield_spl_s_standard_01_mk1_macro.xml,20106,1124,split,1,703,67,8.90,0
38,SPL S Shield Generator Mk2,shield_spl_s_standard_01_mk2_macro.xml,20106,1134,split,2,840,103,8.90,0
39,SPL S Shield Generator Mk3,shield_spl_s_standard_01_mk3_macro.xml,20106,1144,split,3,1200,177,8.90,0
40,SPL XL Shield Generator Mk1,shield_spl_xl_standard_01_mk1_macro.xml,20106,4044,split,1,110058,398,0.00,9000


###  Двигатели <a class="anchor" id="id_3b"></a>

Адрес в descriptions_eng:  
```<page id="20107" title="Engines Thrusters" descr="Names and descriptions of ship engines" voice="no">```

```<macros>
  <macro name="engine_arg_l_allround_01_mk1_macro" class="engine">
    <component ref="engine_arg_l_allround_01_mk1" />
    <properties>
      <identification name="{20107,3004}" basename="{20107,3001}" shortname="{20107,3005}" makerrace="argon" description="{20107,3002}" mk="1" />
      <boost duration="29" thrust="2" attack="10" release="1" />
      <travel charge="20" thrust="31" attack="75" release="22.5" />
      <thrust forward="4206" reverse="4627" />
      <angular />
      <hull max="4033" threshold="0.3" />
      <effects>
        <boosting ref="arg_boost_fx_l" />
      </effects>
      <sounds>
        <enginedetail ref="enginedetail_ship_l_01" />
      </sounds>
    </properties>
  </macro>
</macros>```

Данные Behemot Vanguard, экипированного 'ARG L All-round Engine Mk1':  
speed 127     
acceleration 64    
boost speed 255  
travel speed 3951 

Данные Behemot Vanguard, ничего не экипировано:  
acceleration = 10 

Acceleration = 10 дефолтное ускорение

In [141]:
# Базовая версия корабля (ничего не экипировано)
ship_data[ship_data['name_id']=='11002']

,component,filename,ship_class,explosiondamage_shield,explosiondamage_value,storage_missile,storage_unit,hull,secrecy_level,purpose,people_capacity,ship_mass,inertia_pitch,inertia_roll,inertia_yaw,drag_forward,drag_reverse,drag_horizontal,drag_vertical,drag_pitch,drag_yaw,drag_roll,ship_type,name,name_id,primary_shield_slots,turret_shield_slots,l_turret_slots,m_turret_slots,engines_slots
0,ship_arg_l_destroyer_01,ship_arg_l_destroyer_01_a_macro,ship_l,1000,5000,160,10,93000,2,fight,44,196.016,96.271,96.271,77.016,99.004,396.016,73.005,73.005,106.203,106.203,106.203,destroyer,Behemoth Vanguard,11002,3,9,2,8,3


In [142]:
##find name
#ex = descriptions_Sobject.find('t', attrs = {'id':1})
ex = descriptions_Sobject.find('page', attrs = {'id':20107}).find('t',attrs = {'id':3004})
#type(ex.contents)
name = re.search('[^()]+',ex.text).group()
name

'ARG L All-round Engine Mk1'

In [143]:
##find name
#ex = descriptions_Sobject.find('t', attrs = {'id':1})
ex = descriptions_Sobject.find('page', attrs = {'id':20107}).find('t',attrs = {'id':3124})
#type(ex.contents)
name = re.search('[^()]+',ex.text).group()
name

'BOR L All-round Engine Mk1'

thruster example:     
```<macros>
  <macro name="thruster_gen_l_allround_01_mk1_macro" class="engine">
    <component ref="thruster_gen_l_allround_01_mk1" />
    <properties>
      <identification name="{20107,12004}" basename="{20107,12001}" shortname="{20107,12005}" description="{20107,12002}" mk="1" type="thrustertypes" />
      <component virtual="1" />
      <thrust strafe="775.774" pitch="666.62" yaw="920.014" roll="717.299" />
      <angular roll="20" pitch="60" />
      <hull integrated="1" />
    </properties>
  </macro>
</macros>```

In [144]:
##find name
ex = descriptions_Sobject.find('page', attrs = {'id':20107}).find('t',attrs = {'id':12004})
name = re.search('[^()]+',ex.text).group()
name

'L All-round Thrusters Mk1'

Данные Behemot Vanguard, экипированного L All-round Thrusters Mk1:  
acceleration = 10(m/s)  
strafe acceleration 10.0    
yaw (degrees/sec) = 8.7  
pitch (degrees/sec) = 6.3    
roll (degrees/sec) = 6.8  

Данные самого ускорителя:
```  
  <thrust strafe="775.774" 
  pitch="666.62"  
  yaw="920.014"    
  roll="717.299" />
  <angular roll="20" pitch="60" />
  ```

In [145]:
# Путь к каталогу с двигателями
cat_engines  = r'h:/Steam/X4/Unpacked/assets/props/Engines/macros'
# Путь к каталогу с ускорителями
cat_thrusters = r'h:/Steam/X4/Unpacked/assets/props/Engines/macros'
#dlc ships shields 
split_engines = r'h:\Steam\X4\Unpacked\extensions\split_dlc\assets\props\Engines\macros'
avarice_engines = r'h:\Steam\X4\Unpacked\extensions\avarice_dlc\assets\props\engines\macros'
terran_engines = r'h:\Steam\X4\Unpacked\extensions\terran_dlc\assets\props\engines\macros'
boron_engines = r'h:\Steam\X4\Unpacked\extensions\boron_dlc\assets\props\engines\macros'

In [146]:
list_of_engine_files = [] #Список нужных файлов с двигателями
# Список для исключения файлов нпс объектов мин, полиции, ракет, дронов, скафандра
keywords_to_ignore = ['_xs_','_spacesuit_','_mine_','_missile_']
# Список каталогов с данными о щитах
engine_folders = [cat_engines,split_engines,avarice_engines,boron_engines,terran_engines ]
for folder in engine_folders:
        for root, dirs, files in os.walk(folder):                
                for file in files:
                        if  (file.startswith("engine_") 
                             and '_video_' not in file 
                             and '_xs_' not in file 
                             and '_spacesuit_' not in file 
                             and '_mine_' not in  file
                             and '_missile_' not in file):                 
                                #print(os.path.join(root, file))
                                list_of_engine_files.append(os.path.join(root, file))
print(len(list_of_engine_files))

132


Пример извлечения данных по одному двигателю

In [147]:
with open(r'h:\Steam\X4\Unpacked\extensions\boron_dlc\assets\props\engines\macros\engine_bor_l_allround_01_mk1_macro.xml', 'r') as user_file:
    file_contents = user_file.read()
#Создание древовидной структуры файла      
engine_Sobject = BeautifulSoup(file_contents, features="xml")

In [148]:
engine_page = descriptions_Sobject.find('page', attrs = {'id':20107})
engine_data = {} # Словарь для сохранения информации о двигателе
# Получение имени файла этого двигателя
try:
    engine_data['filename'] = engine_Sobject.select_one('macro').get('name')+".xml"
    engine_data['class'] = engine_Sobject.select_one('macro').get('class')
    #  page_id и id двигателя из descriptions_eng
    temp =  engine_Sobject.select_one('identification').get('name')
    name = re.search('[^{}]+',temp).group()
    name_list = name.split(',')
    engine_page_id = int(name_list[0]) #page_id
    engine_id = int(name_list[1])       #  id  
    engine_data['page_id'] = engine_page_id  # сохранение page_id в словарь
    engine_data['engine_id'] = engine_id     # сохранение id в словарь
    # параметры boost 
    engine_data['boost_duration'] = engine_Sobject.select_one('boost').get('duration')
    engine_data['boost_thrust'] = engine_Sobject.select_one('boost').get('thrust')
    engine_data['boost_attack'] = engine_Sobject.select_one('boost').get('attack')
    engine_data['boost_release'] = engine_Sobject.select_one('boost').get('release')
    #параметры travel
    engine_data['travel_charge'] = engine_Sobject.select_one('travel').get('charge')
    engine_data['travel_thrust'] = engine_Sobject.select_one('travel').get('thrust')
    engine_data['travel_attack'] = engine_Sobject.select_one('travel').get('attack')
    engine_data['travel_release'] = engine_Sobject.select_one('travel').get('release')
    # параметры thrust
    engine_data['thrust_forward'] = engine_Sobject.select_one('thrust').get('forward')
    engine_data['thrust_reverse'] = engine_Sobject.select_one('thrust').get('reverse')
    # прочность двигателя
    engine_data['hull'] = engine_Sobject.select_one('hull').get('max')
except:
    print('error')
print(engine_data)

{'filename': 'engine_bor_l_allround_01_mk1_macro.xml', 'class': 'engine', 'page_id': 0, 'engine_id': 0, 'boost_duration': '30', 'boost_thrust': '2', 'boost_attack': '10', 'boost_release': '1', 'travel_charge': '15', 'travel_thrust': '30', 'travel_attack': '75', 'travel_release': '23', 'thrust_forward': '4500', 'thrust_reverse': '5000', 'hull': '5000'}


Сбор информации о двигателях

!!!Боронский двигатель в файле 'engine_bor_l_allround_01_mk1_macro.xml' не присутсвует в игре. Вместо него в качестве 'BOR L All-round Engine Mk1' выступает двигатель в файле 'engine_bor_l_travel_01_mk1_macro.xml'

In [149]:
# информация о названиях, она находится в page id = 20107 в переменной descriptions_eng
engine_page = descriptions_Sobject.find('page', attrs = {'id':20107})
engine_info = {} # словарь словарей для сбора информации
#Проход по списку нужных файлов:
for file in list_of_engine_files:    
    with open(file, 'r') as user_file:
        file_contents = user_file.read()
        #Создание древовидной структуры файла      
        engine_Sobject = BeautifulSoup(file_contents, features="xml") 
        engine_data = {} # словарь для сохранения информации о двигателе
        try:
            engine_data['filename'] = engine_Sobject.select_one('macro').get('name')+".xml"
        except:
            print('Error with filename', file)
        try:
            engine_data['class'] = engine_Sobject.select_one('macro').get('class')
        except:
            print("Error with class", filename)
            #  page_id и id двигателя из descriptions_eng
        
        try:
            temp =  engine_Sobject.select_one('identification').get('name')
            name = re.search('[^{}]+',temp).group()
            name_list = name.split(',')
            engine_page_id = int(name_list[0]) #page_id
            engine_id = int(name_list[1])       #  id  
            engine_data['page_id'] = engine_page_id  # сохранение page_id в словарь
            engine_data['engine_id'] = engine_id     # сохранение id в словарь            
            if temp == "{0,0,#'engine_bor_l_allround_01_mk1'}":
                    engine_data['page_id'] = 0  
                    engine_data['engine_id'] = 0                    
        except:
            print("Error with page id", file)               
            # производитель
        try:
            engine_data['maker'] = engine_Sobject.select_one('identification').get('makerrace')
        except:
            print("Error with maker", file)
                #версия
        try:
            engine_data['version'] = engine_Sobject.select_one('identification').get('mk')
        except:
            print("Error with version", file)
                # параметры boost
        try: 
            engine_data['boost_duration'] = engine_Sobject.select_one('boost').get('duration')
            engine_data['boost_thrust'] = engine_Sobject.select_one('boost').get('thrust')
            engine_data['boost_attack'] = engine_Sobject.select_one('boost').get('attack')
            engine_data['boost_release'] = engine_Sobject.select_one('boost').get('release')
        except:
            print('Errors with boost parameters', file)
                #параметры travel
        try:
            engine_data['travel_charge'] = engine_Sobject.select_one('travel').get('charge')
            engine_data['travel_thrust'] = engine_Sobject.select_one('travel').get('thrust')
            engine_data['travel_attack'] = engine_Sobject.select_one('travel').get('attack')
            engine_data['travel_release'] = engine_Sobject.select_one('travel').get('release')
        except:
            print("Errors with travel parameters", file)
            # параметры thrust
        try:
            engine_data['thrust_forward'] = engine_Sobject.select_one('thrust').get('forward')
            engine_data['thrust_reverse'] = engine_Sobject.select_one('thrust').get('reverse')
        except:
            print("Errors with thrust parameters", file)           
            # Сохранение информации в главный словарь под ключом названия двигателя
        try:
            if engine_data['page_id']==0 and engine_data['engine_id']==0:  # в случае {0,0,#'engine_bor_l_allround_01_mk1'}
                continue
            elif engine_data['engine_id']==3124:
                name = 'BOR L All-round Engine Mk1'
                engine_info[name] = engine_data 

            else:        
                tmp = engine_page.find('t',attrs = {'id':engine_data['engine_id']})
                name = re.search('[^()]+',tmp.text).group()   
                engine_info[name] = engine_data           
        except:
            print("Errors while assigning to key in engine_info", file)
            
print(len(engine_info))

130


In [150]:
engines_df = pd.DataFrame.from_dict(engine_info,orient = 'index').reset_index().rename(columns = {'index':'name'})
display(engines_df.head())

,name,filename,class,page_id,engine_id,maker,version,boost_duration,boost_thrust,boost_attack,boost_release,travel_charge,travel_thrust,travel_attack,travel_release,thrust_forward,thrust_reverse
0,ARG L All-round Engine Mk1,engine_arg_l_allround_01_mk1_macro.xml,engine,20107,3004,argon,1,29,2,10,1,20,31,75,22.5,4206,4627
1,ARG L Travel Engine Mk1,engine_arg_l_travel_01_mk1_macro.xml,engine,20107,3044,argon,1,26,2,10,1,20,33,85,37.5,4006,3605
2,ARG M All-round Engine Mk1,engine_arg_m_allround_01_mk1_macro.xml,engine,20107,2004,argon,1,7,8,0.25,1,1,9,30,20,1002,952
3,ARG M All-round Engine Mk2,engine_arg_m_allround_01_mk2_macro.xml,engine,20107,2044,argon,2,7,8,0.25,1,1,9,30,20,1212,1228
4,ARG M All-round Engine Mk3,engine_arg_m_allround_01_mk3_macro.xml,engine,20107,2084,argon,3,7,8,0.25,1,1,9,30,20,1353,1413


In [151]:
engines_df.query('maker=="boron"')

,name,filename,class,page_id,engine_id,maker,version,boost_duration,boost_thrust,boost_attack,boost_release,travel_charge,travel_thrust,travel_attack,travel_release,thrust_forward,thrust_reverse
100,BOR L All-round Engine Mk1,engine_bor_l_travel_01_mk1_macro.xml,engine,20107,3124,boron,1,34,2,14,0.04,0,45,178.5,1.25,3004,2704
101,BOR M All-round Engine Mk1,engine_bor_m_allround_01_mk1_macro.xml,engine,20107,2814,boron,1,9,11.2,0.35,0.04,0,13,42,1,751,714
102,BOR M All-round Engine Mk2,engine_bor_m_allround_01_mk2_macro.xml,engine,20107,2824,boron,2,9,11.2,0.35,0.04,0,13,42,1,909,921
103,BOR M All-round Engine Mk3,engine_bor_m_allround_01_mk3_macro.xml,engine,20107,2834,boron,3,9,11.2,0.35,0.04,0,13,42,1,1015,1060
104,BOR S All-round Engine Mk1,engine_bor_s_allround_01_mk1_macro.xml,engine,20107,1804,boron,1,9,11.2,0.35,0.04,0,20,42,1,297,312
105,BOR S All-round Engine Mk2,engine_bor_s_allround_01_mk2_macro.xml,engine,20107,1814,boron,2,9,11.2,0.35,0.04,0,20,42,1,359,430
106,BOR S All-round Engine Mk3,engine_bor_s_allround_01_mk3_macro.xml,engine,20107,1824,boron,3,9,11.2,0.35,0.04,0,20,42,1,401,509
107,BOR XL All-round Engine Mk1,engine_bor_xl_travel_01_mk1_macro.xml,engine,20107,4124,boron,1,34,2,14,0.04,0,45,178.5,1.25,7912,7120


Проверка на наличие диблирующихся названий:

In [152]:
display(engines_df.groupby('name')['name'].value_counts())

name
ARG L All-round Engine Mk1     1
ARG L Travel Engine Mk1        1
ARG M All-round Engine Mk1     1
ARG M All-round Engine Mk2     1
ARG M All-round Engine Mk3     1
                              ..
XEN L All-round Engine Mk1     1
XEN M Combat Engine Mk1        1
XEN M Travel Engine Mk1        1
XEN S Combat Engine Mk1        1
XEN XL All-round Engine Mk1    1
Name: count, Length: 130, dtype: int64

Проверка на пропуски в таблицах после внесения изменений

In [153]:
missing_values_tab(engines_df)
engines_df.info()

Ваш датафрейм содержит 17 колонок и 130 строк.

Он имеет  0 колонок с пропущенными значениями.


,Пропущенные значения,% от всех значений,Тип данных


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   name            130 non-null    object
 1   filename        130 non-null    object
 2   class           130 non-null    object
 3   page_id         130 non-null    int64 
 4   engine_id       130 non-null    int64 
 5   maker           130 non-null    object
 6   version         130 non-null    object
 7   boost_duration  130 non-null    object
 8   boost_thrust    130 non-null    object
 9   boost_attack    130 non-null    object
 10  boost_release   130 non-null    object
 11  travel_charge   130 non-null    object
 12  travel_thrust   130 non-null    object
 13  travel_attack   130 non-null    object
 14  travel_release  130 non-null    object
 15  thrust_forward  130 non-null    object
 16  thrust_reverse  130 non-null    object
dtypes: int64(2), object(15)
memory usage: 17.4+ KB


Изменение типа колонок

In [154]:
#Изменение типа колонок на int
cols_to_int = ['boost_duration','travel_charge','travel_thrust','thrust_forward', 'thrust_reverse']
cols_to_float = ['boost_thrust','boost_attack','boost_release', 'travel_attack', 'travel_release']
try:
    for column in cols_to_int:               
        engines_df[column] = engines_df[column].astype('int')
    for column in cols_to_float:               
        engines_df[column] = engines_df[column].astype('float')
except BaseException:
    print(column)
engines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            130 non-null    object 
 1   filename        130 non-null    object 
 2   class           130 non-null    object 
 3   page_id         130 non-null    int64  
 4   engine_id       130 non-null    int64  
 5   maker           130 non-null    object 
 6   version         130 non-null    object 
 7   boost_duration  130 non-null    int32  
 8   boost_thrust    130 non-null    float64
 9   boost_attack    130 non-null    float64
 10  boost_release   130 non-null    float64
 11  travel_charge   130 non-null    int32  
 12  travel_thrust   130 non-null    int32  
 13  travel_attack   130 non-null    float64
 14  travel_release  130 non-null    float64
 15  thrust_forward  130 non-null    int32  
 16  thrust_reverse  130 non-null    int32  
dtypes: float64(5), int32(5), int64(2), 

## Сохранение таблиц <a class="anchor" id="id_4"></a>

Сохранение таблицы кораблей

In [155]:
#Копия таблицы
saveCopy_df = ship_data.copy(deep = True)
# Сохранение в формате csv
saveCopy_df.to_csv(save_folder + r'\raw_X4data.csv', index=False)
#Сохранение в формате excel
saveCopy_df.to_excel(save_folder + r'\raw_X4data.xlsx',  sheet_name='ship_data',index=False)

Сохранение таблицы щитов

In [156]:
#Копия таблицы
shields_saveCopy_df = shields_df.copy(deep = True)
# Сохранение в формате csv
shields_saveCopy_df.to_csv(save_folder + r'\shields_X4data.csv', index=False)
#Сохранение в формате excel
shields_saveCopy_df.to_excel(save_folder + r'\shields_X4data.xlsx',  sheet_name='shield_data',index=False)

Сохранение таблицы двигателей

In [157]:
#Копия таблицы
engines_saveCopy_df = engines_df.copy(deep = True)
# Сохранение в формате csv
engines_saveCopy_df.to_csv(save_folder + r'\engines_X4data.csv', index=False)
#Сохранение в формате excel
engines_saveCopy_df.to_excel(save_folder + r'\engines_X4data.xlsx',  sheet_name='engine_data',index=False)